# New York AirBnB

__Import statements__

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import (
    train_test_split,
    cross_validate
)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn import set_config
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingRegressor

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.feature_selection import RFE, RFECV

import eli5

from sklearn.feature_selection import SequentialFeatureSelector

### Importing the dataset

In [42]:
data = pd.read_csv("../data/raw/airbnb.csv")

/home/utkarshsaboo45/miniconda3/envs/env_airbnb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [43]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,48,2019-11-04,0.33,3,322,0,NaN
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,1,408,2021-06-29,4.91,1,220,38,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2016-06-05,0.53,2,365,0,NaN
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,5,2,2021-08-08,0.02,1,91,1,NaN
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,505,2021-10-20,3.70,1,218,31,NaN


In [44]:
# Drop license, id, last_review
# data.drop(columns=["id", "license", "last_review"], inplace=True)

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37713 entries, 0 to 37712
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37713 non-null  int64  
 1   name                            37700 non-null  object 
 2   host_id                         37713 non-null  int64  
 3   host_name                       37682 non-null  object 
 4   neighbourhood_group             37713 non-null  object 
 5   neighbourhood                   37713 non-null  object 
 6   latitude                        37713 non-null  float64
 7   longitude                       37713 non-null  float64
 8   room_type                       37713 non-null  object 
 9   price                           37713 non-null  int64  
 10  minimum_nights                  37713 non-null  int64  
 11  number_of_reviews               37713 non-null  int64  
 12  last_review                     

In [46]:
data.dropna(subset=["reviews_per_month"], inplace=True)

In [47]:
data["last_review_year"] = pd.DatetimeIndex(data["last_review"]).year
data["last_review_month"] = pd.DatetimeIndex(data["last_review"]).month
data["last_review_date"] = pd.DatetimeIndex(data["last_review"]).day

In [48]:
# Splitting the data

In [49]:
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=123)

In [50]:
## Some Stupidity

# def find_no_of_listings(data):
#     temp_df = pd.DataFrame(data["host_id"].value_counts()).reset_index()

#     temp_df.rename(columns={
#         "index": "host_id",
#         "host_id": "number_of_listings"
#     }, inplace=True)

#     data = pd.merge(data, temp_df, on="host_id")

#     data["number_of_listings"] = pd.cut(
#         data.number_of_listings,
#         bins=[0, 1, 10, np.inf],
#         labels=["1", "2-10", ">10"]
#     )

#     return data

In [51]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [52]:
# X_train["neighbourhood"].value_counts()#.to_csv("apple.csv")

In [53]:
# # Converting the neighbourhoods occuring less than 86 times to "Others"
# less_frequent_list = X_train["neighbourhood"].value_counts()[86:].index.tolist()
# X_train.loc[X_train.query(
#     "neighbourhood in @less_frequent_list"
# ).index, "neighbourhood"] = "Others"

In [54]:
# X_train["neighbourhood"].value_counts()

In [55]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,last_review_year,last_review_month,last_review_date
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,...,48,2019-11-04,0.33,3,322,0,NaN,2019,11,4
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,...,408,2021-06-29,4.91,1,220,38,NaN,2021,6,29
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,...,50,2016-06-05,0.53,2,365,0,NaN,2016,6,5
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,...,2,2021-08-08,0.02,1,91,1,NaN,2021,8,8
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,...,505,2021-10-20,3.70,1,218,31,NaN,2021,10,20


In [56]:
numeric_features = [
    "latitude",
    "longitude",
    "price",
    "minimum_nights",
    "calculated_host_listings_count",
    "availability_365",
    "last_review_year",
    "last_review_date",
    "id",
    "host_id",
    "number_of_reviews",
    "number_of_reviews_ltm"
]

categorical_features = [
    "neighbourhood_group",
    "room_type",
    "last_review_month"
]

text_features = [
    "name"
]

drop_features = [
    "license",
    "host_name",
    "last_review",
    "neighbourhood"
]

passthrough_features = [

]

target_column = "reviews_per_month"

assert (
    len(numeric_features) +
    len(categorical_features) +
    len(text_features) +
    len(drop_features) +
    len(passthrough_features) +
    len([target_column])
) == len(train_data.columns)

In [85]:
X_train, y_train = train_data.drop(columns=[target_column]), train_data[target_column]
X_val, y_val = val_data.drop(columns=[target_column]), val_data[target_column]
X_train_val, y_train_val = train_val_data.drop(columns=[target_column]), train_val_data[target_column]

In [58]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [59]:
function_transformer = FunctionTransformer(
    np.reshape, kw_args={"newshape": -1}
)

pipe_text_feats = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    function_transformer,
    CountVectorizer(stop_words="english", max_features=30)
)

column_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), categorical_features),
    (pipe_text_feats, text_features),
    ("drop", drop_features),
    ("passthrough", passthrough_features)
)

In [60]:
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['latitude', 'longitude', 'price',
                                  'minimum_nights',
                                  'calculated_host_listings_count',
                                  'availability_365', 'last_review_year',
                                  'last_review_date', 'id', 'host_id',
                                  'number_of_reviews',
                                  'number_of_reviews_ltm']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['neighbourhoo...
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('functiontransformer',
               

In [61]:
new_columns = (
    numeric_features +
    column_transformer.named_transformers_["onehotencoder"].get_feature_names_out().tolist() +
    column_transformer.named_transformers_["pipeline"].named_steps["countvectorizer"].get_feature_names_out().tolist() +
    passthrough_features
)

In [62]:
pd.DataFrame(column_transformer.transform(X_train), columns=new_columns)

,latitude,longitude,price,minimum_nights,calculated_host_listings_count,availability_365,last_review_year,last_review_date,id,host_id,...,new,nyc,park,private,room,spacious,studio,sunny,village,williamsburg
0,-0.406307,0.111532,-0.088612,0.399872,-0.259772,-0.237283,0.764204,-0.942082,0.240064,-0.705988,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.960255,-0.597747,-0.051700,-0.550828,-0.259772,-0.903438,0.764204,1.002941,1.336932,-0.740110,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,-0.122256,-0.881149,-0.002484,-0.664912,5.811509,-0.266247,0.764204,0.183984,1.530824,0.467355,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.575909,-0.306592,-0.113220,0.399872,-0.155693,-0.135912,0.764204,0.286353,1.274427,2.118255,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.913452,1.029390,-0.338795,-0.626884,-0.190386,0.204406,0.764204,-1.453930,-0.186715,0.087669,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17095,1.229243,-0.304653,-0.379808,-0.702940,-0.259772,-0.903438,0.764204,-0.839712,1.434540,-0.406440,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
17096,0.790320,0.746570,-0.404416,0.323816,-0.225079,-0.686214,0.764204,0.695832,1.487830,2.372057,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17097,-0.617706,0.056480,-0.474139,-0.702940,0.052466,1.167433,0.764204,0.491093,0.817329,-0.768836,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
17098,1.412290,-0.017181,-0.310085,-0.626884,-0.259772,-0.816548,-0.314539,0.183984,0.818855,-0.257824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# Credits to Varada K.

# def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
#     """
#     Returns mean and std of cross validation

#     Parameters
#     ----------
#     model :
#         scikit-learn model
#     X_train : numpy array or pandas DataFrame
#         X in the training data
#     y_train :
#         y in the training data

#     Returns
#     ----------
#         pandas Series with mean scores from cross_validation
#     """

#     scores = cross_validate(model, X_train, y_train, **kwargs)

#     mean_scores = pd.DataFrame(scores).mean()
#     std_scores = pd.DataFrame(scores).std()
#     out_col = []

#     for i in range(len(mean_scores)):
#         out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

#     return pd.Series(data=out_col, index=mean_scores.index)

In [64]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score):

    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_train": model.score(X_train, y_train),
        "r2_test": model.score(X_val, y_val),
        "mse_train": mean_squared_error(y_train, y_train_pred),
        "mse_test": mean_squared_error(y_val, y_val_pred),
        "mape_train": mean_absolute_percentage_error(y_train, y_train_pred),
        "mape_test": mean_absolute_percentage_error(y_val, y_val_pred)
    }
    scores_result = pd.Series(score_dict)
    return scores_result

In [82]:
# Adapted from Lecture Notes

def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))


# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer,
}

### Baseline Regression

In [66]:
results = {}

In [27]:
models = {
    "Dummy": make_pipeline(
        column_transformer, DummyRegressor()
    ),
    "Ridge": make_pipeline(
        column_transformer, Ridge()
    ),
    "Random_Forest_reg": make_pipeline(
        column_transformer, RandomForestRegressor(random_state=123)
    ),
    "XGBoost_reg": make_pipeline(
        column_transformer, XGBRegressor(verbosity=0)
    ),
    "lgbm_reg": make_pipeline(
        column_transformer, LGBMRegressor()
    ),
    "catBoost_reg": make_pipeline(
        column_transformer, CatBoostRegressor(verbose=0)
    )
}

for model_name, model in models.items():
    results[model_name] = cross_val_scores(
        model,
        X_train,
        y_train,
        X_val,
        y_val,
        return_train_score=True
    )

In [28]:
pd.DataFrame(results)

,Dummy,Ridge,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg
r2_train,0.000000,0.389500,0.926435,0.958081,0.795359,0.875625
r2_test,-0.000010,0.484823,0.500858,0.433224,0.504185,0.553416
mse_train,18.968049,11.579991,1.395379,0.795113,3.881636,2.359146
mse_test,17.962028,9.253537,8.965511,10.180341,8.905761,8.021479
mape_train,14.434554,4.038569,0.144263,0.603201,0.669306,0.841592
mape_test,14.117369,3.872633,0.413916,0.915507,0.704105,0.916343


## Feature Selection

In [80]:
rfecv = RFECV(Ridge(), min_features_to_select=40, n_jobs=-1)

pipe_catboost_rfecv = make_pipeline(
    column_transformer, rfecv, CatBoostRegressor(random_state=42, verbose=False)
)

results["catboost_rfecv"] = cross_val_scores(
    pipe_catboost_rfecv, X_train, y_train, X_val, y_val, return_train_score=True
)

In [77]:
pd.DataFrame(results)

,catboost_rfecv
r2_train,0.868417
r2_test,0.547076
mse_train,2.495879
mse_test,8.135359
mape_train,0.897401
mape_test,0.963658


In [78]:
pipe_catboost_rfecv.named_steps["rfecv"].n_features_

59

In [ ]:
results[model_name] = cross_val_scores(
    model,
    X_train,
    y_train,
    X_val,
    y_val,
    return_train_score=True
)

## Hyperparam Tuning

In [86]:
params = {
    "catboostregressor__iterations": [100, 150, 200],
    "catboostregressor__learning_rate": [0.03, 0.1],
    "catboostregressor__depth": [2, 4, 6, 8],
    "catboostregressor__l2_leaf_reg": [0.2, 0.5, 1, 3]
}

In [98]:
pipe_catboost = make_pipeline(column_transformer, CatBoostRegressor(verbose=False))

random_search = RandomizedSearchCV(
    pipe_catboost,
    params,
    n_jobs=-1,
    n_iter=30,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [99]:
random_search.fit(X_train_val, y_train_val)

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'price',
                                                                                'minimum_nights',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365',
                                                                                'last_review_year',
                                                     

In [100]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_fit_time",
        "mean_score_time",
        "param_catboostregressor__iterations",
        "param_catboostregressor__learning_rate",
        "param_catboostregressor__depth",
        "param_catboostregressor__l2_leaf_reg",
        "mean_train_neg RMSE",
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by="mean_test_r2", ascending=False).T

,12,7,29,6,15,10,0,16,26,2,...,19,21,11,8,13,25,17,1,18,24
mean_fit_time,6.82749,7.159252,4.519613,14.295692,11.474975,9.86927,6.577761,8.492657,3.71702,6.519207,...,2.289982,11.406348,4.198781,2.825403,6.21152,3.966341,6.493315,3.766759,2.92752,2.810925
mean_score_time,0.5618,0.391883,0.1514,0.619772,0.61807,0.385799,0.199729,0.593557,0.428624,0.419583,...,0.351579,0.632619,0.123512,0.301817,0.392555,0.458726,0.537961,0.743606,0.442195,0.549462
param_catboostregressor__iterations,150,150,150,200,150,200,150,100,100,150,...,100,150,100,100,200,200,200,150,100,150
param_catboostregressor__learning_rate,0.03,0.1,0.03,0.03,0.03,0.03,0.1,0.03,0.03,0.1,...,0.03,0.1,0.03,0.1,0.1,0.1,0.1,0.1,0.1,0.03
param_catboostregressor__depth,8,6,6,8,8,8,6,8,6,6,...,4,8,4,2,4,2,6,2,2,2
param_catboostregressor__l2_leaf_reg,0.2,1,1,0.5,0.5,3,3,1,0.5,0.2,...,0.2,0.2,3,1,0.2,1,0.5,0.5,0.5,3
mean_train_neg RMSE,-2.327018,-2.187393,-2.635367,-2.250473,-2.364878,-2.528047,-2.376882,-2.586144,-2.725816,-2.055575,...,-2.867279,-1.680025,-2.969064,-2.894005,-2.322776,-2.77745,-1.926418,-2.813035,-2.878335,-3.074582
std_train_neg RMSE,0.048267,0.035014,0.057869,0.043179,0.044653,0.047988,0.045437,0.056073,0.066231,0.025196,...,0.093151,0.019378,0.098548,0.103911,0.054152,0.083983,0.024845,0.091856,0.102866,0.110936
mean_test_mape,-128.663083,-84.726162,-148.567686,-92.089974,-132.756374,-136.160584,-83.817898,-231.200762,-243.91595,-83.65932,...,-276.042316,-82.903006,-333.60675,-139.640137,-91.526469,-142.740791,-87.568691,-133.913598,-131.890998,-282.356861
mean_train_mape,-126.581714,-81.551232,-147.146382,-89.25289,-130.491512,-133.646785,-82.588326,-229.684249,-242.449326,-79.448097,...,-275.057704,-74.957212,-332.275303,-139.208397,-88.992919,-141.647131,-82.670398,-133.234069,-131.639441,-280.671418


## Stacking

### Hyperparameter Tuning

In [29]:
#Hyperparameter tune the Ridge

In [30]:
param_grid = {'ridge__alpha': 10.0 ** np.arange(-2, 6, 1)}

In [31]:
pipe_lr = make_pipeline(column_transformer, Ridge())

random_search = RandomizedSearchCV(
    pipe_lr,
    param_grid,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit='r2'
)

NameError: name 'scoring_metrics' is not defined

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_fit_time", 
        "mean_score_time", 
        "param_ridge__alpha", 
        "mean_train_neg RMSE", 
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

In [ ]:
pipe_lr_final = make_pipeline(column_transformer, Ridge(alpha=1000))
pipe_lr_final.fit(X_train, y_train)

eli5.explain_weights(pipe_lr_final.named_steps["ridge"], feature_names=new_columns)

In [ ]:
data = {
    "Importance": pipe_lr_final.named_steps["ridge"].coef_,
}
pd.DataFrame(data=data, index=new_columns,).sort_values(
    by="Importance", ascending=False
)[:10]